In [55]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from collections import Counter
from dotenv import load_dotenv
import os
import json
from chromadb.config import Settings
from chromadb.utils import embedding_functions
import chromadb
import openai
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff
load_dotenv()

True

In [56]:
def parse_detection_result(detection_result):

    detection_result = detection_result.lower()
    result=""
    if "yes" in detection_result:
        result = "yes"
    elif "no" in detection_result:
        result = "no"
    else: 
        print("error: ", detection_result)

    return result

In [57]:

#input_path = "/home/akazad/test_tool_perfapim/src/result/cpp_detection_result_stratified_ctx_t3_f3.json" for llm only
input_path = "result/cpp_detection_result_stratified_ctx_t3_f5.json" # for llm + embedding
output_path = "result/cpp_detection_result_yes_stratified_ctx_t3_f5.json"

In [58]:
# read the file
with open(input_path, encoding="utf-8") as f:
    data = f.readlines()
    data = [line for line in data if line != "\n"]
    data = [json.loads(line) for line in data]

In [59]:
print("data length: ", len(data))

data length:  67


In [60]:

res_list = []
for idx, line in enumerate(data):
    result = line["detection_result"]
    print(type(result))
    print("=====================================")
    print(result)   
    result = parse_detection_result(result) 
    data[idx]["result"] = result
    data[idx]["label"] = ""
    if type(line["code_before"])==str:
        line["code_before"] = line["code_before"].split("\n") # convert string to list
    #line["example"] = line["example"].split("\n")
    line["detection_result"] = line["detection_result"].split("\n")
    res_list.append(line)

<class 'str'>
Yes
<class 'str'>
Yes
<class 'str'>
Yes
<class 'str'>
No
<class 'str'>
Yes
<class 'str'>
No
<class 'str'>
Yes
<class 'str'>
No
<class 'str'>
No
<class 'str'>
Yes
<class 'str'>
Yes
<class 'str'>
Yes
<class 'str'>
No
<class 'str'>
Yes
<class 'str'>
Yes
<class 'str'>
No
<class 'str'>
No
<class 'str'>
No
<class 'str'>
Yes
<class 'str'>
No
<class 'str'>
No
<class 'str'>
No
<class 'str'>
Yes
<class 'str'>
Yes
<class 'str'>
No
<class 'str'>
Yes
<class 'str'>
No
<class 'str'>
No
<class 'str'>
No
<class 'str'>
Yes
<class 'str'>
No
<class 'str'>
No
<class 'str'>
No
<class 'str'>
No
<class 'str'>
No
<class 'str'>
Yes
<class 'str'>
Yes
<class 'str'>
No
<class 'str'>
No
<class 'str'>
No
<class 'str'>
Yes
<class 'str'>
No
<class 'str'>
No
<class 'str'>
No
<class 'str'>
Yes
<class 'str'>
Yes
<class 'str'>
No
<class 'str'>
No
<class 'str'>
Yes
<class 'str'>
Yes
<class 'str'>
Yes
<class 'str'>
No
<class 'str'>
Yes
<class 'str'>
No
<class 'str'>
No
<class 'str'>
No
<class 'str'>
No
<class 

In [61]:
len(res_list)

67

In [62]:
result_list = [x["result"] for x in res_list ]
counter = Counter(result_list)
print(counter)
yes_list = [x for x in res_list if x["result"] == "yes"]

Counter({'no': 39, 'yes': 28})


In [63]:
len(yes_list)

28

In [64]:
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(yes_list, f, indent=4)

In [50]:
no_list = [x for x in res_list if x["result"] == "no"]

In [65]:
len(no_list)

30

In [66]:
path = "result/cpp_detection_result_no_stratified_ctx_t3_f5.json"

In [67]:
with open(path, "w", encoding="utf-8") as f:
    json.dump(no_list, f, indent=4)